In [ ]:
cd G:\sp5 2022 Diali

In [1]:
import os
import harp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import cartopy.crs as ccrs
from cmcrameri import cm
import requests
import sentinelsat
import shutil
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
filename = "S5P_OFFL_L2__SO2____20221018T065817_20221018T083946_25976_03_020401_20221020T044200.nc"

In [ ]:
# api = sentinelsat.SentinelAPI('s5pguest', 's5pguest', 'https://s5phub.copernicus.eu/dhus', show_progressbars=False)
# result = api.download_all(api.query(filename=filename))

In [ ]:
product = harp.import_product(filename)

In [ ]:
print(product)

In [ ]:
print(product.SO2_column_number_density)

In [ ]:
print(type(product.SO2_column_number_density.data))
print(product.SO2_column_number_density.data.shape)

# Plotting

In [ ]:
SO2val = product.SO2_column_number_density.data
SO2units = product.SO2_column_number_density.unit
SO2description = product.SO2_column_number_density.description

latc=product.latitude.data
lonc=product.longitude.data

colortable=cm.batlow
vmin=0
vmax=0.0001

In [ ]:
fig=plt.figure(figsize=(20, 10))
ax = plt.axes(projection=ccrs.PlateCarree())


img = plt.scatter(lonc, latc, c=SO2val,
                vmin=vmin, vmax=vmax, cmap=colortable, s=1, transform=ccrs.PlateCarree())

ax.coastlines()

cbar = fig.colorbar(img, ax=ax, orientation='horizontal', fraction=0.04, pad=0.1)
cbar.set_label(f'{SO2description} [{SO2units}]')
cbar.ax.tick_params(labelsize=14)
plt.show()

# Applying operations when importing data with HARP

In [ ]:
operations = ";".join([
    "latitude>-20;latitude<40",
    "SO2_column_number_density_validity>50",
    "keep(datetime_start,scan_subindex,latitude,longitude,SO2_column_number_density)",
    "derive(SO2_column_number_density [DU])",
])

print(type(operations))
print(operations)

In [ ]:
reduced_product = harp.import_product(filename, operations)

In [ ]:
print(reduced_product)

In [ ]:
SO2val = reduced_product.SO2_column_number_density.data
SO2units = reduced_product.SO2_column_number_density.unit
SO2description = reduced_product.SO2_column_number_density.description

latc=reduced_product.latitude.data
lonc=reduced_product.longitude.data

colortable=cm.batlow
# For Dobson Units
vmin=0
vmax=8

In [ ]:
SO2val.max()

In [ ]:
fname= r'F:\Indian Shape Files\Indian Administrative bourders\IND_adm\IND_adm1.shp'
adm1_shapes = list(shpreader.Reader(fname).geometries())
fname2 = r'F:\Indian Shape Files\Only Indian Boundaries\India_Boundary\India_Boundary.shp'
adm1_shapes2 = list(shpreader.Reader(fname2).geometries())
fname3 = r'F:\Clip-data-from-netCDF-file-and-plot-using-Cartopy-main\Dharashiv.shp'
adm1_shapes3 = list(shpreader.Reader(fname3).geometries())

fig=plt.figure(figsize=(20, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([68, 98, 8, 38], ccrs.PlateCarree())

img = plt.scatter(lonc, latc, c=SO2val,
                vmin=0, vmax=2, cmap='cool', s=1, transform=ccrs.PlateCarree())

ax.coastlines()
ax.add_geometries(adm1_shapes, ccrs.PlateCarree(),
                  edgecolor='black', alpha=0.5,lw = 2, facecolor='none') #
ax.add_geometries(adm1_shapes2, ccrs.PlateCarree(),
                  edgecolor='black', alpha=0.5,lw = 1, facecolor='none') #
ax.add_geometries(adm1_shapes3, ccrs.PlateCarree(),
                  edgecolor='black', alpha=0.5,lw = 1, facecolor='none') #
ax.gridlines()
cbar = fig.colorbar(img, ax=ax, orientation='horizontal', fraction=0.04, pad=0.1)
cbar.set_label(f'{SO2description} [{SO2units}]')
cbar.ax.tick_params(labelsize=14)
plt.show()

In [ ]:
cd C:\Users\IITM\Downloads

In [ ]:
filename_pattern = "S5P_OFFL_L2__SO2____20221017T.nc"

In [ ]:
filename = "S5P_OFFL_L2__SO2____20221027T054743_20221027T072913_26103_03_020401_20221029T033931.nc"
uvai_data = harp.import_product(filename)

In [ ]:
print(uvai_data)

In [ ]:
print(uvai_data.SO2_column_number_density_validity)

In [ ]:
operations = ";".join([
    "SO2_column_number_density_validity>50",
    "keep(latitude_bounds,longitude_bounds,datetime_start,datetime_length,SO2_column_number_density)",
    "derive(datetime_stop {time} [days since 2000-01-01])",
    "derive(datetime_start [days since 2000-01-01])",
    "exclude(datetime_length)",
    "bin_spatial(721,-90,0.25,1441,-180,0.25)", # lat_cells, corner_lat, lat_resolution, lon_cells, corner_lon, lon_resolution, 
    "derive(SO2_column_number_density [DU])",
    "derive(latitude {latitude})",
    "derive(longitude {longitude})",
])

In [ ]:
reduce_operations = "squash(time, (latitude, longitude, latitude_bounds, longitude_bounds));bin()"

In [ ]:
filenames = r"S5P_OFFL_L2__SO2____20221024T*.nc"

In [ ]:
merged = harp.import_product(filenames, operations, reduce_operations=reduce_operations)

In [ ]:
print(merged)

In [ ]:
gridlat = np.append(merged.latitude_bounds.data[:,0], merged.latitude_bounds.data[-1,1])
gridlon = np.append(merged.longitude_bounds.data[:,0], merged.longitude_bounds.data[-1,1])

In [ ]:
SO2val = merged.SO2_column_number_density.data
SO2units = merged.SO2_column_number_density.unit
SO2description = merged.SO2_column_number_density.description

colortable=cm.batlow
# For Dobson Units
vmin=0
vmax=1

In [ ]:
fname= r'F:\Indian Shape Files\Indian Administrative bourders\IND_adm\IND_adm1.shp'
adm1_shapes = list(shpreader.Reader(fname).geometries())
fname2 = r'F:\Indian Shape Files\Only Indian Boundaries\India_Boundary\India_Boundary.shp'
adm1_shapes2 = list(shpreader.Reader(fname2).geometries())
fname3 = r'F:\Clip-data-from-netCDF-file-and-plot-using-Cartopy-main\Dharashiv.shp'
adm1_shapes3 = list(shpreader.Reader(fname3).geometries())

lon17 = 76.0420
lat17 = 18.1853
def main():
    plt.figure(figsize=(15,12),facecolor = 'white')
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([68,98,8,38], crs=ccrs.PlateCarree()) # [lonmin,lonmax,latmin,latmax]
#     ax.coastlines()
    ax.add_geometries(adm1_shapes, ccrs.PlateCarree(),
                  edgecolor='red', alpha=0.5,lw = 2, facecolor='none') #
    ax.add_geometries(adm1_shapes2, ccrs.PlateCarree(),
                  edgecolor='red', alpha=0.5,lw = 2, facecolor='none') #
#     ax.add_geometries(adm1_shapes3, ccrs.PlateCarree(),
#                   edgecolor='red', alpha=0.5,lw = 2, facecolor='none') #
#     ax.add_feature(cfeature.LAND, edgecolor='black')
#     ax.add_feature(cfeature.BORDERS,lw = 2,edgecolor='red')
#     ax.add_feature(cfeature.STATES.with_scale('10m'),
#                linestyle='-', alpha=.25, facecolor='none',lw = 2, edgecolor='red')
    
#     ax.add_feature(cfeature.COASTLINE)
#     ax.gridlines()
    ax.set_facecolor('grey')
    ax.set_title('Sulphur dioxide (SO2) from TROPOMI S5P on 24-10-2022' ,
                  fontweight="bold", size=20,family= 'Arial')
    filled_c = plt.pcolormesh(gridlon, gridlat, SO2val[0,:,:], vmin=vmin, vmax=vmax, cmap='jet', transform=ccrs.PlateCarree())
    
    ax.plot(lon23102022,lat23102022,markersize=10,marker='^',color='yellow',lw=4,label='23102022')
    ax.plot(lon24102022,lat24102022,markersize=10,marker='^',color='lime',lw=4,label='24102022')
#     plt.text(76.0420,18.4,'Osmanabad Airport',horizontalalignment='right', color= 'red',fontweight="bold", size=20,family= 'Arial')
    ax.legend()
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlocator = mticker.FixedLocator(np.arange(68,98,4))
    gl.ylocator = mticker.FixedLocator(np.arange(8,38,4))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 12,'color': 'red', 'weight': 'bold'}
    gl.ylabel_style = {'size': 12,'color': 'red', 'weight': 'bold'}
        # Add a colorbar for the filled contour.
    a = plt.colorbar(filled_c,ax=ax, orientation='vertical',pad=0.05,shrink=0.75)
    a.set_label(label=f'{SO2description}[DU]',weight='bold', fontsize='15')
#     plt.figtext(0.25, 0.085, "\xa9 kmmraoiitm@gmail.com", ha="center", 
#                 fontsize=15)#box={"facecolor": "green", "alpha": 0.75})

if __name__ == '__main__':
    main()
plt.savefig('SO2_TROPOMI_S5P_on_24102022.jpg',bbox_inches='tight',dpi=720)
plt.show()

# Cyclone Satrang

In [ ]:
sat = pd.read_excel(r'C:\Users\IITM\Desktop\Sitrang\sitrang_cYCLONE.xlsx')

In [ ]:
ti23102022 = sat['ISO_TIME'][0:4]
lat23102022 = sat['LAT/degrees_north'][0:4]
lon23102022 = sat['LON/degrees_east'][0:4]
ti24102022 = sat['ISO_TIME'][4:]
lat24102022 = sat['LAT/degrees_north'][4:]
lon24102022 = sat['LON/degrees_east'][4:]

In [ ]:
ti23102022

In [ ]:
sat['ISO_TIME']

# NO2

In [2]:
cd C:\Users\IITM\Downloads

C:\Users\IITM\Downloads


In [3]:
filename = "S5P_OFFL_L2__NO2____20221017T021258_20221017T035427_25959_03_020400_20221018T180242.nc"
trop_no2 = harp.import_product(filename)

In [4]:
print(trop_no2)

source product = 'S5P_OFFL_L2__NO2____20221017T021258_20221017T035427_25959_03_020400_20221018T180242.nc'

int scan_subindex {time=1877400}
double datetime_start {time=1877400} [seconds since 2010-01-01]
float datetime_length [s]
long orbit_index
long validity {time=1877400}
float latitude {time=1877400} [degree_north]
float longitude {time=1877400} [degree_east]
float latitude_bounds {time=1877400, 4} [degree_north]
float longitude_bounds {time=1877400, 4} [degree_east]
float sensor_latitude {time=1877400} [degree_north]
float sensor_longitude {time=1877400} [degree_east]
float sensor_altitude {time=1877400} [m]
float solar_zenith_angle {time=1877400} [degree]
float solar_azimuth_angle {time=1877400} [degree]
float sensor_zenith_angle {time=1877400} [degree]
float sensor_azimuth_angle {time=1877400} [degree]
double pressure_bounds {time=1877400, vertical=34, 2} [Pa]
float tropospheric_NO2_column_number_density {time=1877400} [mol/m^2]
float tropospheric_NO2_column_number_density_uncer

In [5]:
print(trop_no2.tropospheric_NO2_column_number_density )

type = float
dimension = {time=1877400}
unit = 'mol/m^2'
valid_min = -inf
valid_max = inf
description = 'tropospheric vertical column of NO2'
data =
[nan nan nan ... nan nan nan]



In [6]:
operations = ";".join([
    "tropospheric_NO2_column_number_density_validity>75",
    "keep(latitude_bounds,longitude_bounds,datetime_start,datetime_length,tropospheric_NO2_column_number_density )",
    "derive(datetime_stop {time} [days since 2000-01-01])",
    "derive(datetime_start [days since 2000-01-01])",
    "exclude(datetime_length)",
    "bin_spatial(721,-90,0.25,1441,-180,0.25)", # lat_cells, corner_lat, lat_resolution, lon_cells, corner_lon, lon_resolution, 
    "derive(tropospheric_NO2_column_number_density [Pmolec/cm2])",
    "derive(latitude {latitude})",
    "derive(longitude {longitude})",
])


In [7]:
reduce_operations = "squash(time, (latitude, longitude, latitude_bounds, longitude_bounds));bin()"

In [8]:
filenames = r"S5P_OFFL_L2__NO2____20221023T*.nc"

In [9]:
merged = harp.import_product(filenames, operations, reduce_operations=reduce_operations)

CLibraryError: [HDF5] H5C__load_entry(): incorrect metadata checksum after all read attempts (major="Object cache", minor="Read failed") (D:\bld\hdf5_split_1658932175228\work\src\H5C.c:7315)

In [ ]:
gridlat = np.append(merged.latitude_bounds.data[:,0], merged.latitude_bounds.data[-1,1])
gridlon = np.append(merged.longitude_bounds.data[:,0], merged.longitude_bounds.data[-1,1])

In [ ]:
NO2val = merged.tropospheric_NO2_column_number_density.data
NO2units = merged.tropospheric_NO2_column_number_density.unit
NO2description = merged.tropospheric_NO2_column_number_density.description

colortable=cm.batlow
# For Dobson Units
vmin=0
vmax=8

In [ ]:
fname= r'F:\Indian Shape Files\Indian Administrative bourders\IND_adm\IND_adm1.shp'
adm1_shapes = list(shpreader.Reader(fname).geometries())
fname2 = r'F:\Indian Shape Files\Only Indian Boundaries\India_Boundary\India_Boundary.shp'
adm1_shapes2 = list(shpreader.Reader(fname2).geometries())
fname3 = r'F:\Clip-data-from-netCDF-file-and-plot-using-Cartopy-main\Dharashiv.shp'
adm1_shapes3 = list(shpreader.Reader(fname3).geometries())

lon17 = 76.0420
lat17 = 18.1853
def main():
    plt.figure(figsize=(15,12),facecolor = 'white')
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([68,98,8,38], crs=ccrs.PlateCarree()) # [lonmin,lonmax,latmin,latmax]
#     ax.coastlines()
    ax.add_geometries(adm1_shapes, ccrs.PlateCarree(),
                  edgecolor='red', alpha=0.5,lw = 2, facecolor='none') #
    ax.add_geometries(adm1_shapes2, ccrs.PlateCarree(),
                  edgecolor='red', alpha=0.5,lw = 2, facecolor='none') #
#     ax.add_geometries(adm1_shapes3, ccrs.PlateCarree(),
#                   edgecolor='red', alpha=0.5,lw = 2, facecolor='none') #
#     ax.add_feature(cfeature.LAND, edgecolor='black')
#     ax.add_feature(cfeature.BORDERS,lw = 2,edgecolor='red')
#     ax.add_feature(cfeature.STATES.with_scale('10m'),
#                linestyle='-', alpha=.25, facecolor='none',lw = 2, edgecolor='red')
    
#     ax.add_feature(cfeature.COASTLINE)
#     ax.gridlines()
    ax.set_facecolor('grey')
    ax.set_title('Tropospheric NO2 from TROPOMI S5P on 23-10-2022' ,
                  fontweight="bold", size=20,family= 'Arial')
    filled_c = plt.pcolormesh(gridlon, gridlat, NO2val[0,:,:], vmin=vmin, vmax=vmax, cmap='jet', transform=ccrs.PlateCarree())
    
    ax.plot(lon23102022,lat23102022,markersize=10,marker='^',color='yellow',lw=4,label='23102022')
#     ax.plot(lon24102022,lat24102022,markersize=10,marker='^',color='lime',lw=4,label='24102022')
#     plt.text(76.0420,18.4,'Osmanabad Airport',horizontalalignment='right', color= 'red',fontweight="bold", size=20,family= 'Arial')
    ax.legend()
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlocator = mticker.FixedLocator(np.arange(68,98,4))
    gl.ylocator = mticker.FixedLocator(np.arange(8,38,4))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 12,'color': 'red', 'weight': 'bold'}
    gl.ylabel_style = {'size': 12,'color': 'red', 'weight': 'bold'}
        # Add a colorbar for the filled contour.
    a = plt.colorbar(filled_c,ax=ax, orientation='vertical',pad=0.05,shrink=0.75)
    a.set_label(label=f'{NO2description}[Pmol/cm2]',weight='bold', fontsize='15')
#     plt.figtext(0.25, 0.085, "\xa9 kmmraoiitm@gmail.com", ha="center", 
#                 fontsize=15)#box={"facecolor": "green", "alpha": 0.75})

if __name__ == '__main__':
    main()
plt.savefig('Trop_NO2_TROPOMI_S5P_on_23102022.jpg',bbox_inches='tight',dpi=720)
plt.show()

In [11]:
import h5py 